In [1]:
#resnet 是imagenet 2015冠军，
#残差网络，基于残差块的堆叠， residual block

In [7]:
import tensorflow as tf
import cifar10_input
batch_size = 64
images_trains, labels_trains = cifar10_input.distorted_inputs(data_dir="../cifar_data/cifar-10-batches-bin/",
                                                              batch_size=batch_size)
images_test, labels_test = cifar10_input.inputs(eval_data=True,data_dir="../cifar_data/cifar-10-batches-bin/",
                                               batch_size=batch_size)

import tensorflow.contrib.slim as slim

Filling queue with 20000 CIFAR images before starting to train. This will take a few minutes.


In [9]:
#下采样函数
def sub_sample(x, factor, scope=None):
    if factor == 1:
        return x
    return slim.max_pool2d(x,[1,1], scope=scope)

In [10]:
def residual_block(x, bottleneck_depth, out_depth, stride=1, scope='residual_block'):
    in_depth = x.get_shape().as_list()[-1]
    with tf.variable_scope(scope):
        if in_depth == out_depth:
            shortcut = sub_sample(x, stride, 'shortcut')
        else:
            shortcut = slim.conv2d(x,out_depth,[1,1], stride=stride, activation_fn=None,scope='shortcut')
        residual = slim.conv2d(x,bottleneck_depth, [1,1], stride=1, scope='conv1')
        residual = slim.conv2d(residual, bottleneck_depth, 3, stride, scope='conv2')
        residual = slim.conv2d(residual, out_depth, [1,1], stride=1, activation_fn=None, scope='conv3')
        
        output = tf.nn.relu(shortcut+residual)
        return output

In [12]:
def resnet(inputs, num_classes, reuse=None, is_training=None, verbose=False):
    with tf.variable_scope('resnet',reuse=reuse):
        net = inputs
        if verbose:
            print('input: {}'.format(net.shape))
        with slim.arg_scope([slim.conv2d, slim.max_pool2d, slim.avg_pool2d], padding='SAME'):
            with tf.variable_scope('block1'):
                net = slim.conv2d(net, 32, [5,5], stride=2, scope='conv_5x5')
                if verbose:
                    print('block1: {}'.format(net.shape))
            with tf.variable_scope('blcok2'):
                net = slim.max_pool2d(net, [3,3], 2, scope='max_pool')
                net = residual_block(net, 32, 128, scope='residual_block1')
                net = residual_block(net, 32, 128, scope='residual_block2')
                if verbose:
                    print('block2: {}'.format(net.shape))
            with tf.variable_scope('block3'):
                net = residual_block(net, 64, 256, stride=2, scope='residual_block1')
                net = residual_block(net, 64, 256, scope='residual_block2')
                if verbose:
                    print('block3: {}'.format(net.shape))
                
            with tf.variable_scope('block4'):
                net = residual_block(net, 128, 512, stride=2, scope='residual_block1')
                net = residual_block(net, 128, 512, scope='residual_block2')
                if verbose:
                    print('block4: {}'.format(net.shape))
            with tf.variable_scope('classification'):
                net = tf.reduce_mean(net, [1,2], name='global_pool', keep_dims=True)
                net = slim.flatten(net, scope='flattern')
                net = slim.fully_connected(net, num_classes, activation_fn=None, 
                                           normalizer_fn=None, scope='logit')
                if verbose:
                    print('classification: {}'.format(net.shape))
            return net

In [13]:
with slim.arg_scope([slim.conv2d], activation_fn = tf.nn.relu, normalizer_fn = slim.batch_norm) as sc:
    conv_scope = sc

is_training = tf.placeholder(tf.bool, name='is_training')
image_holder = tf.placeholder(shape=[batch_size,24,24,3], dtype=tf.float32)
label_holder = tf.placeholder(shape=[batch_size], dtype=tf.int32)

with slim.arg_scope(conv_scope):
    train_out = resnet(image_holder,10, is_training=is_training, verbose=True)

with tf.variable_scope('loss'):
    train_loss = tf.reduce_mean(tf.losses.sparse_softmax_cross_entropy(logits=train_out,
                                                                       labels=label_holder, scope='train'))

with tf.variable_scope('accuracy'):
    with tf.name_scope('train'):
        train_acc = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(train_out, axis=-1,output_type=tf.int32), 
                                                    label_holder), tf.float32))

lr = 1e-3
opt = tf.train.MomentumOptimizer(learning_rate=lr, momentum=0.9)
update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
with tf.control_dependencies(update_ops):
    train_op = opt.minimize(train_loss)

sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())
tf.train.start_queue_runners()

for i in range(1000):
    image_bath, label_batch = sess.run([images_trains, labels_trains])
    _, loss1, acc1 = sess.run([train_op,train_loss,train_acc], feed_dict={image_holder: image_bath, label_holder: label_batch, is_training:True})
    if i % 10 ==0:
        print("Step%d  train loss %.6f train acc: %.6f" % (i+1, loss1, acc1))

input: (64, 24, 24, 3)
block1: (64, 12, 12, 32)
block2: (64, 6, 6, 128)
block3: (64, 3, 3, 256)
block4: (64, 2, 2, 512)
Instructions for updating:
keep_dims is deprecated, use keepdims instead
classification: (64, 10)
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Step1  train loss 3.270705 train acc: 0.062500
Step11  train loss 2.590307 train acc: 0.078125
Step21  train loss 2.765359 train acc: 0.062500
Step31  train loss 2.308105 train acc: 0.125000
Step41  train loss 2.310807 train acc: 0.140625
Step51  train loss 2.176364 train acc: 0.203125
Step61  train loss 2.193724 train acc: 0.156250
Step71  train loss 2.117293 train acc: 0.234375
Step81  train loss 2.089896 train acc: 0.281250
Step91  train loss 2.057678 train acc: 0.265625
Step101  train loss 2.137844 train acc: 0.234375
Step111  train loss 2.160743 train acc: 0.234375
Step121  train loss 1.942740 train acc: 0.296875
Step131  train loss 2.040877 train acc: 0.312500
Step141  train loss 1.95

KeyboardInterrupt: 